In [1]:
import pyslammer as slam

In [6]:
histories = slam.sample_ground_motions()
ky = 0.02
motion = histories["Chi-Chi_1999_TCU068-090"]
dt = motion[0][1] - motion[0][0]

da = slam.Decoupled()
out = da.decoupled(ain_p=motion[1],
             uwgt_p=20.0,
             height_p=50.0,
             vs_p=600.0,
             damp1_p=0.05,
             refstrain_p=0.01,
             dt_p=dt,
             scal_p=1.0,
             g_p=9.80665,
             vr_p=600.0,
             ca=[[0.1],[ky]],
             dv3_p=False)

In [7]:
da

In [14]:
a = max(da.avgacc)

In [15]:
b = max(motion[1])

In [17]:
b/a

In [11]:
da.qq